# Experiment 003: Remove Data Leakage

**Objective:** Remove the data leakage from 'relative_position' feature and measure true model performance.

**Issue identified:** In exp_002, 'relative_position' = position / notebook_size, where 'position' is the target variable. This is data leakage.

**Changes:**
- Remove 'relative_position' feature from training
- Keep all other features (TF-IDF, headings, notebook-level stats)
- Use same 5-fold GroupKFold validation

**Expected outcome:** CV score will drop significantly, revealing true model performance.

In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from tqdm import tqdm
import re
from collections import Counter
from sklearn.model_selection import GroupKFold
from sklearn.metrics import make_scorer
from scipy.stats import kendalltau
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# Set paths
TRAIN_PATH = Path('/home/data/train')
TEST_PATH = Path('/home/data/test')
ORDERS_PATH = Path('/home/data/train_orders.csv')

print("Loading data...")
orders_df = pd.read_csv(ORDERS_PATH)
print(f"Orders shape: {orders_df.shape}")
print(f"Unique notebooks: {orders_df['id'].nunique()}")

Loading data...


Orders shape: (119256, 2)
Unique notebooks: 119256


In [2]:
# Load a subset of training data for faster iteration
# Using 5,000 notebooks like exp_002
np.random.seed(42)
all_notebooks = orders_df['id'].unique()
selected_notebooks = np.random.choice(all_notebooks, size=5000, replace=False)

print(f"Selected {len(selected_notebooks)} notebooks for training")

Selected 5000 notebooks for training


In [3]:
# Feature extraction functions
def extract_basic_features(df):
    """Extract basic text statistics"""
    df['source_length'] = df['source'].str.len()
    df['line_count'] = df['source'].str.count('\\n') + 1
    df['word_count'] = df['source'].str.split().str.len()
    df['char_count'] = df['source'].str.len()
    
    # Binary flags
    df['has_import'] = df['source'].str.contains('import\\s+\\w+', regex=True, na=False).astype(int)
    df['has_comment'] = df['source'].str.contains('#', na=False).astype(int)
    df['has_heading'] = df['source'].str.contains('^#+\\s+', regex=True, na=False).astype(int)
    df['has_code_block'] = df['source'].str.contains('```', na=False).astype(int)
    df['has_link'] = df['source'].str.contains('\\[.*\\]\\(.*\\)', regex=True, na=False).astype(int)
    
    return df

def extract_heading_features(df):
    """Extract heading-related features"""
    # Extract heading level (1-6)
    df['heading_level'] = 0
    for level in range(1, 7):
        mask = df['source'].str.match(f'^#{{{level}}}\\s+', na=False)
        df.loc[mask, 'heading_level'] = level
    
    # Binary flags for common heading texts
    common_headings = ['introduction', 'conclusion', 'summary', 'results', 'methods', 
                       'analysis', 'eda', 'exploratory', 'data', 'preprocessing',
                       'model', 'training', 'evaluation', 'references', 'appendix',
                       'setup', 'imports', 'installation', 'requirements',
                       'visualization', 'plot', 'train', 'test', 'validation']
    
    for heading in common_headings:
        df[f'heading_{heading}'] = df['source'].str.contains(heading, case=False, na=False).astype(int)
    
    return df

def extract_semantic_features(df):
    """Extract semantic position features"""
    df['has_print'] = df['source'].str.contains('print\\s*\\(', na=False).astype(int)
    df['has_kaggle'] = df['source'].str.contains('kaggle', case=False, na=False).astype(int)
    df['has_input'] = df['source'].str.contains('input', case=False, na=False).astype(int)
    df['has_data'] = df['source'].str.contains('\\bdata\\b', case=False, na=False).astype(int)
    df['has_function'] = df['source'].str.contains('def\\s+\\w+\\s*\\(', regex=True, na=False).astype(int)
    df['has_class'] = df['source'].str.contains('class\\s+\\w+', regex=True, na=False).astype(int)
    df['has_model'] = df['source'].str.contains('\\bmodel\\b', case=False, na=False).astype(int)
    df['has_train'] = df['source'].str.contains('\\btrain\\b', case=False, na=False).astype(int)
    df['has_test'] = df['source'].str.contains('\\btest\\b', case=False, na=False).astype(int)
    df['has_plot'] = df['source'].str.contains('\\.plot\\s*\\(|\\.show\\s*\\(', regex=True, na=False).astype(int)
    df['has_import'] = df['source'].str.contains('^\\s*import\\s+|^\\s*from\\s+', regex=True, na=False).astype(int)
    
    return df

In [4]:
# Load and process training data
print("Loading training notebooks...")
train_data = []
notebook_sizes = {}

for notebook_id in tqdm(selected_notebooks, desc="Processing notebooks"):
    notebook_path = TRAIN_PATH / f"{notebook_id}.json"
    
    if not notebook_path.exists():
        continue
    
    with open(notebook_path, 'r') as f:
        notebook = json.load(f)
    
    cell_order = orders_df[orders_df['id'] == notebook_id]['cell_order'].iloc[0].split()
    cell_positions = {cell_id: pos for pos, cell_id in enumerate(cell_order)}
    
    notebook_sizes[notebook_id] = len(cell_order)
    
    # The notebook structure has cell_type and source as separate dictionaries
    cell_types = notebook['cell_type']
    sources = notebook['source']
    
    for cell_id in cell_order:
        cell_type = 1 if cell_types[cell_id] == 'code' else 0
        source = sources[cell_id]
        
        train_data.append({
            'notebook_id': notebook_id,
            'cell_id': cell_id,
            'cell_type': cell_type,
            'source': source,
            'position': cell_positions[cell_id]
        })

train_df = pd.DataFrame(train_data)
print(f"Training data shape: {train_df.shape}")
print(f"Average cells per notebook: {train_df.groupby('notebook_id').size().mean():.1f}")

Loading training notebooks...


Processing notebooks:   0%|          | 0/5000 [00:00<?, ?it/s]

Processing notebooks:   0%|          | 11/5000 [00:00<00:49, 100.99it/s]

Processing notebooks:   0%|          | 22/5000 [00:00<00:48, 101.79it/s]

Processing notebooks:   1%|          | 33/5000 [00:00<00:48, 101.86it/s]

...

Processing notebooks:  99%|█████████▉| 4974/5000 [00:50<00:00, 93.59it/s]

Processing notebooks: 100%|█████████▉| 4984/5000 [00:50<00:00, 94.11it/s]

Processing notebooks: 100%|█████████▉| 4994/5000 [00:50<00:00, 92.32it/s]

Processing notebooks: 100%|██████████| 5000/5000 [00:50<00:00, 98.07it/s]

Training data shape: (232291, 5)
Average cells per notebook: 46.5


In [5]:
# Extract features
print("Extracting features...")
train_df = extract_basic_features(train_df)
train_df = extract_heading_features(train_df)
train_df = extract_semantic_features(train_df)

# Calculate notebook-level statistics (WITHOUT relative_position to avoid leakage)
notebook_stats = train_df.groupby('notebook_id').agg({
    'source_length': ['mean', 'std'],
    'word_count': ['mean', 'std'],
    'cell_type': 'mean',  # code ratio (was incorrectly named cell_type_code)
    'position': 'max'  # notebook size
}).round(2)

notebook_stats.columns = ['_'.join(col).strip() for col in notebook_stats.columns]
notebook_stats = notebook_stats.reset_index()
notebook_stats.rename(columns={'position_max': 'notebook_size', 'cell_type_mean': 'cell_type_code_mean'}, inplace=True)

# Merge notebook-level features
train_df = train_df.merge(notebook_stats, on='notebook_id', how='left')

# NOTE: DELIBERATELY NOT adding relative_position to avoid data leakage
# train_df['relative_position'] = train_df['position'] / train_df['notebook_size']

print(f"Final training shape: {train_df.shape}")
print(f"Notebook-level features added: {list(notebook_stats.columns)}")

Extracting features...


Final training shape: (232291, 55)
Notebook-level features added: ['notebook_id', 'source_length_mean', 'source_length_std', 'word_count_mean', 'word_count_std', 'cell_type_code_mean', 'notebook_size']


In [6]:
# Prepare TF-IDF features
print("Preparing TF-IDF features...")
# Get all markdown cells for TF-IDF
markdown_cells = train_df[train_df['cell_type'] == 0]['source'].fillna('').tolist()

# Fit TF-IDF vectorizer on markdown content
vectorizer = TfidfVectorizer(
    max_features=1000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.9
)

vectorizer.fit(markdown_cells)
tfidf_feature_names = [f'tfidf_{i}' for i in range(len(vectorizer.get_feature_names_out()))]

print(f"TF-IDF features created: {len(tfidf_feature_names)}")

# Transform all cells (both code and markdown)
all_texts = train_df['source'].fillna('').tolist()
tfidf_matrix = vectorizer.transform(all_texts)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_feature_names)

# Concatenate with main dataframe
train_df = pd.concat([train_df.reset_index(drop=True), tfidf_df], axis=1)
print(f"Final shape with TF-IDF: {train_df.shape}")

Preparing TF-IDF features...


TF-IDF features created: 1000


Final shape with TF-IDF: (232291, 1055)


In [13]:
# Define feature columns (WITHOUT relative_position)
basic_features = ['source_length', 'line_count', 'word_count', 'char_count',
                  'has_import', 'has_comment', 'has_heading', 'has_code_block', 'has_link']

heading_features = ['heading_level'] + [f'heading_{h}' for h in ['introduction', 'conclusion', 'summary', 'results', 'methods', 
                       'analysis', 'eda', 'exploratory', 'data', 'preprocessing',
                       'model', 'training', 'evaluation', 'references', 'appendix',
                       'setup', 'imports', 'installation', 'requirements',
                       'visualization', 'plot', 'train', 'test', 'validation']]

semantic_features = ['has_print', 'has_kaggle', 'has_input', 'has_data', 'has_function',
                     'has_class', 'has_model', 'has_train', 'has_test', 'has_plot']  # Removed 'has_import' to avoid duplication

notebook_features = ['source_length_mean', 'source_length_std', 'word_count_mean', 
                     'word_count_std', 'cell_type_code_mean', 'notebook_size']

# TF-IDF features
tfidf_features = [f'tfidf_{i}' for i in range(1000)]

# Combine all features
feature_cols = basic_features + heading_features + semantic_features + notebook_features + tfidf_features

print(f"Total features: {len(feature_cols)}")
print(f"Basic features: {len(basic_features)}")
print(f"Heading features: {len(heading_features)}")
print(f"Semantic features: {len(semantic_features)}")
print(f"Notebook features: {len(notebook_features)}")
print(f"TF-IDF features: {len(tfidf_features)}")

Total features: 1050
Basic features: 9
Heading features: 25
Semantic features: 10
Notebook features: 6
TF-IDF features: 1000


In [14]:
# Cross-validation setup
def kendall_tau_score(y_true, y_pred):
    """Calculate Kendall tau correlation"""
    return kendalltau(y_true, y_pred)[0]

gkf = GroupKFold(n_splits=5)
groups = train_df['notebook_id']

X = train_df[feature_cols]
y = train_df['position']

print("Starting cross-validation...")
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Train model
    model = lgb.LGBMRegressor(
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_pred = model.predict(X_val)
    
    # Calculate per-notebook Kendall tau
    val_df = train_df.iloc[val_idx].copy()
    val_df['pred_position'] = y_pred
    
    fold_scores = []
    for notebook_id in val_df['notebook_id'].unique():
        notebook_data = val_df[val_df['notebook_id'] == notebook_id]
        if len(notebook_data) > 1:
            score = kendall_tau_score(
                notebook_data['position'].values,
                notebook_data['pred_position'].values
            )
            fold_scores.append(score)
    
    fold_score = np.mean(fold_scores)
    cv_scores.append(fold_score)
    print(f"Fold {fold + 1}: {fold_score:.4f}")

print(f"\nCV Score: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")
print(f"Individual folds: {[f'{score:.4f}' for score in cv_scores]}")

Starting cross-validation...


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.269372 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 184292
[LightGBM] [Info] Number of data points in the train set: 185833, number of used features: 1045
[LightGBM] [Info] Start training from score 40.378625


Fold 1: 0.3978


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.278287 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 185413
[LightGBM] [Info] Number of data points in the train set: 185833, number of used features: 1044
[LightGBM] [Info] Start training from score 41.840663


Fold 2: 0.4008


...

Fold 3: 0.4011


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.258486 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 185073
[LightGBM] [Info] Number of data points in the train set: 185832, number of used features: 1044
[LightGBM] [Info] Start training from score 41.894507


Fold 4: 0.3991


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.257136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 185596
[LightGBM] [Info] Number of data points in the train set: 185834, number of used features: 1045
[LightGBM] [Info] Start training from score 41.895773


Fold 5: 0.4080

CV Score: 0.4014 ± 0.0035
Individual folds: ['0.3978', '0.4008', '0.4011', '0.3991', '0.4080']


In [15]:
# Train final model on all data
print("Training final model on all training data...")
final_model = lgb.LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

final_model.fit(X, y)

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 most important features:")
print(feature_importance.head(20).to_string(index=False))

Training final model on all training data...


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.334277 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 191007
[LightGBM] [Info] Number of data points in the train set: 232291, number of used features: 1048
[LightGBM] [Info] Start training from score 41.576471



Top 20 most important features:
            feature  importance
      notebook_size        1900
      source_length         297
    word_count_mean         168
 source_length_mean         162
      heading_model         143
cell_type_code_mean         132
         word_count         130
  source_length_std         116
         line_count         114
       heading_test         106
     word_count_std          87
          tfidf_245          86
          tfidf_858          71
          tfidf_328          68
          tfidf_684          65
          tfidf_213          63
         char_count          62
       heading_data          59
      heading_level          58
          tfidf_943          56


In [34]:
# Generate predictions for test set
print("Generating predictions for test set...")
test_notebooks = list(TEST_PATH.glob('*.json'))
print(f"Total test notebooks: {len(test_notebooks)}")

def predict_notebook_order(notebook_path, model, vectorizer, feature_cols):
    """Predict cell order for a single notebook"""
    with open(notebook_path, 'r') as f:
        notebook = json.load(f)
    
    notebook_id = notebook_path.stem
    cell_types = notebook['cell_type']
    sources = notebook['source']
    cell_ids = list(cell_types.keys())
    
    # Create features dataframe
    features_df = pd.DataFrame({
        'cell_id': cell_ids,
        'cell_type': [1 if cell_types[cid] == 'code' else 0 for cid in cell_ids],
        'source': [sources[cid] for cid in cell_ids]
    })
    
    # Extract features
    features_df = extract_basic_features(features_df)
    features_df = extract_heading_features(features_df)
    features_df = extract_semantic_features(features_df)
    
    # Calculate notebook-level statistics
    notebook_stats = features_df.agg({
        'source_length': ['mean', 'std'],
        'word_count': ['mean', 'std'],
        'cell_type': 'mean'
    }).round(2)
    
    notebook_stats.columns = ['_'.join(col).strip() for col in notebook_stats.columns]
    notebook_stats = notebook_stats.reset_index()
    
    # Rename columns to match training
    notebook_stats = notebook_stats.rename(columns={
        'index': 'agg_func',
        'source_length_mean': 'source_length_mean',
        'source_length_std': 'source_length_std', 
        'word_count_mean': 'word_count_mean',
        'word_count_std': 'word_count_std',
        'cell_type_mean': 'cell_type_code_mean'
    })
    
    # Add notebook_size (number of cells)
    notebook_size = len(features_df)
    
    # Add notebook-level features to each row
    features_df['source_length_mean'] = notebook_stats.loc[0, 'source_length_mean']
    features_df['source_length_std'] = notebook_stats.loc[0, 'source_length_std']
    features_df['word_count_mean'] = notebook_stats.loc[0, 'word_count_mean']
    features_df['word_count_std'] = notebook_stats.loc[0, 'word_count_std']
    features_df['cell_type_code_mean'] = notebook_stats.loc[0, 'cell_type_code_mean']
    features_df['notebook_size'] = notebook_size
    
    # Transform TF-IDF features
    all_texts = features_df['source'].fillna('').tolist()
    tfidf_matrix = vectorizer.transform(all_texts)
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f'tfidf_{i}' for i in range(tfidf_matrix.shape[1])])
    
    # Concatenate TF-IDF features
    features_df = pd.concat([features_df.reset_index(drop=True), tfidf_df], axis=1)
    
    # Select only the features used in training
    X = features_df[feature_cols]
    
    # Predict positions
    predictions = model.predict(X)
    
    # Order cells by predicted position
    features_df['predicted_position'] = predictions
    ordered_cells = features_df.sort_values('predicted_position')['cell_id'].tolist()
    
    return ' '.join(ordered_cells)

Generating predictions for test set...
Total test notebooks: 20000


In [35]:
# Generate predictions for all test notebooks
submission_data = []

for notebook_path in tqdm(test_notebooks, desc="Predicting test notebooks"):
    try:
        notebook_id = notebook_path.stem
        ordered_cells = predict_notebook_order(notebook_path, final_model, vectorizer, feature_cols)
        
        submission_data.append({
            'id': notebook_id,
            'cell_order': ordered_cells
        })
    except Exception as e:
        print(f"Error with {notebook_id}: {e}")

submission_df = pd.DataFrame(submission_data)
print(f"Submission shape: {submission_df.shape}")
print(submission_df.head())

Predicting test notebooks:   0%|          | 0/20000 [00:00<?, ?it/s]

Predicting test notebooks:   0%|          | 3/20000 [00:00<12:23, 26.89it/s]

Error with a283f88578d447: 'source_length_mean'
Error with 4b5781d84d4f04: 'source_length_mean'
Error with 032e1e8598af69: 'source_length_mean'
Error with 598969c8543546: 'source_length_mean'


Predicting test notebooks:   0%|          | 6/20000 [00:00<12:38, 26.35it/s]

Error with 6986f993569b86: 'source_length_mean'
Error with 34a0ca434fcbd0: 'source_length_mean'


...

Predicting test notebooks: 100%|█████████▉| 19995/20000 [10:56<00:00, 29.46it/s]

Error with bef5b5e0ce1539: 'source_length_mean'
Error with 99706a45df1f86: 'source_length_mean'
Error with 11efadff610e80: 'source_length_mean'
Error with 98d06234743613: 'source_length_mean'
Error with 4b71b0c2e55f36: 'source_length_mean'
Error with 936469b92093e1: 'source_length_mean'
Error with 8f8618901dea8e: 'source_length_mean'


Predicting test notebooks: 100%|█████████▉| 19998/20000 [10:56<00:00, 29.15it/s]

Predicting test notebooks: 100%|██████████| 20000/20000 [10:57<00:00, 30.44it/s]

Error with e7adbec2234af0: 'source_length_mean'
Error with 9bbf276eef11c5: 'source_length_mean'
Error with 08c1e20125251f: 'source_length_mean'
Submission shape: (0, 0)
Empty DataFrame
Columns: []
Index: []


In [36]:
# Save submission
submission_path = '/home/submission/submission_003.csv'
submission_df.to_csv(submission_path, index=False)
print(f"Submission saved to {submission_path}")

# Verify format
sample_sub = pd.read_csv('/home/data/sample_submission.csv')
print(f"\nColumns match: {list(submission_df.columns) == list(sample_sub.columns)}")
print(f"Number of rows match: {len(submission_df) == len(sample_sub)}")

Submission saved to /home/submission/submission_003.csv

Columns match: False
Number of rows match: False


In [11]:
# Debug: Check for duplicate columns
print("Duplicate columns in train_df:")
duplicates = train_df.columns[train_df.columns.duplicated()]
print(duplicates.tolist())

print("\nColumns containing 'has_import':")
has_import_cols = [col for col in train_df.columns if 'has_import' in col]
print(has_import_cols)

print("\nAll columns (first 20):")
print(train_df.columns[:20].tolist())

print("\nShape of train_df:", train_df.shape)

Duplicate columns in train_df:
[]

Columns containing 'has_import':
['has_import']

All columns (first 20):
['notebook_id', 'cell_id', 'cell_type', 'source', 'position', 'source_length', 'line_count', 'word_count', 'char_count', 'has_import', 'has_comment', 'has_heading', 'has_code_block', 'has_link', 'heading_level', 'heading_introduction', 'heading_conclusion', 'heading_summary', 'heading_results', 'heading_methods']

Shape of train_df: (232291, 1055)


In [12]:
# Debug: Check feature_cols list
print("Checking feature_cols for duplicates...")
print(f"Length of feature_cols: {len(feature_cols)}")
print(f"Unique features: {len(set(feature_cols))}")

# Find duplicates in feature_cols
from collections import Counter
counts = Counter(feature_cols)
duplicates = [item for item, count in counts.items() if count > 1]
print(f"\nDuplicate features in feature_cols: {duplicates}")

# Check if 'has_import' is in multiple feature categories
print(f"\nhas_import in basic_features: {'has_import' in basic_features}")
print(f"has_import in semantic_features: {'has_import' in semantic_features}")

# Show the problematic categories
if 'has_import' in basic_features:
    print(f"basic_features: {basic_features}")
if 'has_import' in semantic_features:
    print(f"semantic_features: {semantic_features}")

Checking feature_cols for duplicates...
Length of feature_cols: 1051
Unique features: 1050

Duplicate features in feature_cols: ['has_import']

has_import in basic_features: True
has_import in semantic_features: True
basic_features: ['source_length', 'line_count', 'word_count', 'char_count', 'has_import', 'has_comment', 'has_heading', 'has_code_block', 'has_link']
semantic_features: ['has_print', 'has_kaggle', 'has_input', 'has_data', 'has_function', 'has_class', 'has_model', 'has_train', 'has_test', 'has_plot', 'has_import']
